# RAG pattern with Azure OpenAI and Cognitive Search Vector
This code demonstrates how to use Azure Cognitive Search with OpenAI and Azure Python SDK leveraging the RAG pattern for context specific answers from LLM

## Prerequisites
1. To run the code, install the following packages. Please use the latest pre-release version `pip install azure-search-documents==11.4.0b6`.
2. The [1.0 CreateKnowledgeBaseWithCogSearch notebook](1.0%20CreateKnowledgeBaseWithCogSearch.ipynb) has been run first

- > ! pip install azure-search-documents==11.4.0b6
- > ! pip install openai

## Load all the AOAI API keys and model parameters

In [1]:
import aoai

MY_AOAI_ENDPOINT = 'https://tr-non-prod-gpt4.openai.azure.com/'
MY_AOAI_VERSION = '2023-07-01-preview'
MY_GPT_ENGINE = 'tr-gpt4'
MY_AOAI_EMBEDDING_ENGINE = 'tr-embedding-ada'

status = aoai.setupOpenai(aoai_endpoint=MY_AOAI_ENDPOINT, 
                 aoai_version=MY_AOAI_VERSION)
if status > 0:
    print("AOAI setup succeeded")
else:
    print("AOAI setup failed")


Got OPENAI API Key from environment variable
AOAI setup succeeded


## Authenticate to Azure Cognitive Search and connect

In [2]:
import cog_search

cogSearchCredential = cog_search.getCogSearchCredential()


Got Azure Cognitive Search ADMIN API Key from environment variable


In [3]:
from azure.search.documents.indexes import SearchIndexClient

# The search index
MY_COG_SEARCH_ENDPOINT = 'https://tr-docai-cog-search.search.windows.net'
MY_COG_SEEARCH_INDEX_NAME = 'sample-azure-service-docs-index'

## Import all other necessary search packages

In [4]:
from azure.search.documents import SearchClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    VectorSearchAlgorithmConfiguration,  
)  

## End user question for AOAI

In [5]:
query = "what does scalable storage solution integrate with?" 

## AOAI answer without RAG

In [6]:
my_prompt = [
              {
                "role": "user", 
                "content": f"Answer the user question. \
                            \n\nuser question: {query}\
                            \nanswer:"
                }
            ]      
tokens_used, finish_reason, aoai_answer = aoai.getChatCompletion(the_engine=MY_GPT_ENGINE, 
                                                                           the_messages=my_prompt)
print(f"Tokens: {tokens_used}")
print(f"Finish Reason: {finish_reason}")
print(f"Answer: {aoai_answer}")

Tokens: 85
Finish Reason: stop
Answer: Scalable storage solutions can integrate with various systems such as cloud computing platforms (like AWS, Google Cloud, Azure), databases, content management systems, virtualization platforms, data backup and recovery systems, and various enterprise applications. The integration depends on the specific storage solution and its compatibility.


## AOAI answer with RAG 

## Perform a Hybrid Cognitive Search to get context information

In [7]:
# Hybrid Search
search_client = SearchClient(
                    endpoint=MY_COG_SEARCH_ENDPOINT, 
                    index_name=MY_COG_SEEARCH_INDEX_NAME, 
                    credential=cogSearchCredential)
  
results = search_client.search(  
    search_text=query,  
    vector=aoai.generate_embedding(
                    the_engine=MY_AOAI_EMBEDDING_ENGINE,
                    the_text=query), 
    top_k=1,  
    vector_fields="contentVector",
    select=["title", "content", "category"],
    top=1
)  
  
for result in results:  
    print(f"Title: {result['title']}\n")  
    print(f"Score: {result['@search.score']}\n")  
    print(f"Content: {result['content']}\n")  
    print(f"Category: {result['category']}\n")  


Title: Azure Table Storage

Score: 0.03279569745063782

Content: Azure Table Storage is a fully managed, NoSQL datastore that enables you to store and query large amounts of structured, non-relational data. It provides features like automatic scaling, schema-less design, and a RESTful API. Table Storage supports various data types, such as strings, numbers, and booleans. You can use Azure Table Storage to store and manage your data, build scalable applications, and reduce the cost of your storage. It also integrates with other Azure services, such as Azure Functions and Azure Cosmos DB.

Category: Storage



## Prompt Engineering step to Ground Prompt with the above context

In [8]:
my_context = result['content']
my_prompt = [
              {
                "role": "user", 
                "content": f"Answer the user question from the context provided below. \
                            \n\nuser question: {query}\
                            \ncontext: {my_context}\
                            \nanswer:"
                }
            ]      
tokens_used, finish_reason, aoai_answer = aoai.getChatCompletion(the_engine=MY_GPT_ENGINE, 
                                                                           the_messages=my_prompt)
print(f"Tokens: {tokens_used}")
print(f"Finish Reason: {finish_reason}")
print(f"Answer: {aoai_answer}")


Tokens: 158
Finish Reason: stop
Answer: Azure Table Storage integrates with other Azure services, such as Azure Functions and Azure Cosmos DB.
